<a href="https://colab.research.google.com/github/anlaram/entregable_3_IA/blob/main/instrucciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entregable 3: Un modelo para la clasificación de texto

En este documento vamos a ver cómo podemos utilizar la técnica de transfer-learning para crear un modelo de clasificación de texto, en concreto crearemos un modelo para decir si una valoración de IMDb es positiva o negativa. El contenido de este notebook está basado en gran medida en el libro de [FastAI](https://github.com/fastai/fastbook) y se usa un dataset en inglés, pero las mismas ideas se podrían aplicar para crear modelos en castellano o en otros lenguajes. 

**Importante:** Recuerda tener activado el uso de GPU para el notebook, de lo contrario no podrás entrenar los modelos. 

Para aplicar la técnica de transfer-learning será necesario usar un modelo de lenguaje. Como hemos visto en teoría, un modelo de lenguaje es un modelo encargado de predecir la siguiente palabra de un texto a partir de las que han aparecido atnes. Este tipo de tareas se conoce como *aprendizaje semi-supervisado*: no es necesario proporcionar etiquetas a nuestro modelo, sino simplemente le tenemos que proporcionar grandes cantidades de texto. Este proceso obtiene de manera automática las etiquetas a partir de los datos, y es una tarea que está lejos de ser trivial: advinar la siguiente palabra de un texto requiere una comprensión del lenguaje. Las técnicas de aprendizaje semi-supervisado también pueden ser usadas en otros contextos como la [visión por computador](https://www.fast.ai/2020/01/13/self_supervised/).



Para construir nuestro clasificador de texto, vamos a seguir un proceso, llamado [ULMFit](https://arxiv.org/abs/1801.06146) que consta de tres pasos representados gráficamente en la siguiente figura.

<img alt="Diagram of the ULMFiT process" width="700" caption="The ULMFiT process" id="ulmfit_process" src="https://github.com/joheras/fastbook/blob/master/images/att_00027.png?raw=1">

En primer lugar es necesario disponer de un modelo de lenguaje entrenado con un dataset de tamaño considerable. En nuestro caso la Wikipedia. Este modelo sirve para conocer los fundamentos del lenguaje con el cual se está trabajando. Sin embargo, a la hora de construir un modelo de clasificación es conveniente que el modelo comprenda el estilo que se usa para escribir esos textos. Dicho estilo puede ser más informal o más técnico que el contenido de la wikipedia. En el caso del dataset IMDb, este va a contener una gran cantidad de nombres de directores, actores, y además el estilo de redacción es más informal que los textos que aparecen en la Wikipedia. Por ello, a partir del modelo de lenguaje de Wikipedia entrenaremos un modelo de lenguaje para IMDb, y a partir de ese modelo de lenguaje construiremos nuestro clasificador.

Afortunadamente la librería FastAI ya proporciona un modelo de lenguaje para la Wikipedia (construir este tipo de modelos puede llevar días), por lo que nos podemos centrar en los otros dos pasos. 

## Instalación

Comenzamos instalando las librerías necesarias para este notebook.

In [25]:
!pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.7/dist-packages (2.3.1)


## Modelo de lenguaje para IMDb

Antes de crear nuestro modelo de lenguaje para el dataset de IMDB (o para cualquier otro dataset), tenemos que procesarlo. Este proceso va a constar de 4 pasos:

1. **Tokenización**: convertir el texto en una lista de palabras (o caractéres o n-gramas dependiendo del nivel de granularidad que se quiera obtener).
2. **Numericalización**: construir una lista de todas las palabras que aparecen en el dataset (el vocabulario), y convertir cada palabra en un número mirando el índice que ocupa en el vocabulario. Esta representación es una alternativa al one-hot encoding que vimos en clase. 
3. **Organizar el dataset**: para entrenar un modelo debemos definir cuáles van a ser nuestras entradas y cuales nuestras salidas. Para este tipo de modelos la entrada es una palabra del texto, y la salida es la siguiente palabra del texto. Por ello, nuestra "X" va a ser la secuencia de texto desde la primera palabra hasta la penúltima, y la "Y" será la secuencia de texto desde la segunda palabra hasta la última.
4. **Construir el modelo de lenguaje**: para ello usaremos una red recurrente. 

### Tokenización

A la hora de partir un texto en una lista de palabras hay muchas cuestiones por decidir. Por ejemplo, ¿qué hacemos con la puntuación? ¿Cómo tratamos con contracciones como "don't"? ¿Qué hacemos con las palabras con guiones? Además en ciertos lenguajes como el alemán o el polaco hay palabras muy largas construidas a partir de muchas piezas, y en idiomas como el japones o el chino no hay un concepto de *palabra*. 

Por lo tanto, existen distintas aproximaciones para tokenizar un texto que ya vimos en clase. 

- **Basado en palabras**: se parte el texto basándose en los espacios, y aplicando reglas específicas del lenguaje para separar palabras como "don't" en "do n't". En general, los signos de puntuación también se separan en tokens. 
- **Basado en subpalabras**: se parten las palabras en partes más pequeñas basándose en las partes que aparecen más frecuentemente. Por ejemplo, "occasion" se puede tokenizar en "o c ca sion".
- **Basada en caracteres**: se parte una frase por sus caracteres individuales. 

### Tokenización con fastai

La librería FastAI ofrece su propio tokenizador, vamos a ver cómo usarlo. Pero antes, tenemos que descargar el dataset de IMDB.

In [26]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

Con el siguiente comando podemos ver lo que se ha descargado con el comando anterior.

In [27]:
path.ls()

(#7) [Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/test')]

Como podemos ver se han descargado varias carpetas, de las cuales a nosotros nos van a interesar tres: la carpeta *train* que va a contenter nuestro conjunto de entrenamiento para construir nuestro clasificador, la carpeta *test* que contiene el conjunto de test, y la carpeta *unsup* que contiene valoraciones que no han sido clasificadas. 

Podemos ver además que la carpeta *train* (y lo mismo ocurre para la carpeta *test*) contienen dos subcarpetas, una con las valoraciones positivas y otra con las valoraciones negativas.  

In [28]:
(path/'train').ls()

(#4) [Path('/root/.fastai/data/imdb/train/unsupBow.feat'),Path('/root/.fastai/data/imdb/train/neg'),Path('/root/.fastai/data/imdb/train/pos'),Path('/root/.fastai/data/imdb/train/labeledBow.feat')]

Por último cada una de esas subcarpetas contiene un conjunto de ficheros, y cada fichero contiene una valoración.

In [29]:
(path/'train'/'pos').ls()

(#12500) [Path('/root/.fastai/data/imdb/train/pos/8968_8.txt'),Path('/root/.fastai/data/imdb/train/pos/4680_10.txt'),Path('/root/.fastai/data/imdb/train/pos/12136_9.txt'),Path('/root/.fastai/data/imdb/train/pos/7992_7.txt'),Path('/root/.fastai/data/imdb/train/pos/6006_10.txt'),Path('/root/.fastai/data/imdb/train/pos/9976_7.txt'),Path('/root/.fastai/data/imdb/train/pos/4485_10.txt'),Path('/root/.fastai/data/imdb/train/pos/5534_7.txt'),Path('/root/.fastai/data/imdb/train/pos/7561_9.txt'),Path('/root/.fastai/data/imdb/train/pos/5278_7.txt')...]

Por el momento, para constrir nuestro modelo de lenguaje no es necesario saber si una valoración es positiva o si es negativa, y además nos interesa tener un vocabulario lo más amplio posible, por lo tanto para este modelo de lenguaje vamos a cargar los ficheros que aparecen en las carpetas de *train*, *test* y *unsup*. Para ello vamos a usar la función ``get_text_files`` que carga todos los ficheros que aparecen en una carpeta (incluyendo los que aparecen en subcarpetas).

In [30]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

Ahora podemos mostrar uno de los ficheros que hemos cargado. 

In [31]:
txt = files[0].open().read(); txt[:75]

'If you liked the Grinch movie... go watch that again, because this was no w'

El proceso de tokenizado en FastAI consta de dos pasos. En primer lugar se aplican una serie de reglas para separar las palabras, esto se hace mediante un `WordTokenizer`, dicho objeto se pasa a la clase `Tokenizer` que aplica una serie de reglas adicionales. Vamos a ver como funcionan.

In [32]:
wt = WordTokenizer()
tkn = Tokenizer(wt)

Si aplicamos el `WordTokenizer` a la frase que hemos cargado anteriormente, veremos que el texto se ha separado en una lista de palabras. 

In [33]:
first(wt([txt]))

(#116) ['If','you','liked','the','Grinch','movie','...','go','watch','that'...]

En cambio si aplicamos el `Tokenizer` obtenemos que no solo se separan las palabras sino que también se pasan a minúsculas y aparecen una serie de tokens que empiezan por los caracteres "xx".

In [34]:
coll_repr(tkn(txt), 31)

"(#130) ['xxbos','xxmaj','if','you','liked','the','xxmaj','grinch','movie','…','go','watch','that','again',',','because','this','was','no','where','near','as','good','a','xxmaj','seussian','movie','translation','.','xxmaj','mike'...]"

Los tokens que empiezan por los caracteres "xx" son tokens especiales. Por ejemplo, el token "xxbos" indica el comienzo de un texto. Al reconocer este token de entrada, el modelo será capaz de aprender que necesita "olvidar" todo lo que ha aparecido antes y centrarse en las palabras que aparecen a continuación. Existen múltiples reglas de este estilo proporcionadas por fastai que permiten traducir el texto a una lista de tokens que hacen que sea más sencillo el proceso de aprendizaje. Algunos de esos tokens son los siguientes:

- xxbos:: comienzo de un texto.
- xxmaj:: indica que la siguiente palabra empieza por mayúscula.
- xxunk:: indica que la siguiente palabra es desconocida. 

Una vez que los textos se han partido en tokens, necesitamos convertirlos a números. 

### Numericalización con fastai

La numericalización consiste en asignar a cada token un identificador. Para ello, hay que: 

1. Crear una lista con todas los tokens (el *vocabulario*)
1. Reemplazar cada token con su posición en el vocabulario. 

En la librería FastAI esto se lleva a cabo mediante la clase `Numericalize`. Para llevar a cabo el primer paso debemos constuir nuestro vocabulario (usando las primeras 200 frases de nuestro dataset).

In [35]:
txts = L(o.open().read() for o in files[:2000])
toks200 = txts[:200].map(tkn)
num = Numericalize()
num.setup(toks200)

Podemos ver los primeros 20 elementos de nuestro vocabulario del siguiente modo.

In [36]:
coll_repr(num.vocab,20)

"(#1992) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','and','a','to','of','is','it','i','in'...]"

Notar que los tokens especiales aparecen primero, y a continuación aparecen las palabras de nuestro dataset ordenadas por su frecuencia (es decir, la palabra "the" es la que aparece con más frecuencia en nuestro texto). Por defecto, en `Numericalize` el vocabulario puede contener como máximo 60000 palabras (esto puede cambiarse con el atributo `max_vocab`). Esto significa que el resto de palabras se reemplazan por el token de palabra desconocida representado por `xxunk`. Esto evita un uso excesivo de memoria. Del mismo modo las palabras que aparecen con poca frecuencia son reemplazadas con el token `xxunk` (por defecto, una palabra es considerada como "rara" si aparece menos de tres veces en el dataset, este valor se puede cambiar con el atributo `min_freqs`).

Una vez que se ha creado el objeto `Numericalize` este puede ser usado como una función. 

In [37]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))
nums = num(toks)[:20]
nums

(#130) ['xxbos','xxmaj','if','you','liked','the','xxmaj','grinch','movie','…','go','watch','that','again',',','because','this','was','no','where','near','as','good','a','xxmaj','seussian','movie','translation','.','xxmaj','mike'...]


TensorText([  2,   8,  59,  35, 361,   9,   8,   0,  29,  72, 123, 127,  21, 191,
         11, 108,  20,  25,  64, 162])

También tenemos la operación inversa que convierte los números a palabras.

In [38]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj if you liked the xxmaj xxunk movie … go watch that again , because this was no where'

Ahora que tenemos las palabras representadas como números podemos construir `batches` que serán usados para entrenar nuestro modelo.

### Organizando el dataset en batches

Como hemos comentado anteriormente la "X" de nuestro dataset va a ser la secuencia de palabras (transformadas a números) de la primera a la penúltima palabra del dataset, y la "Y" es la secuencia de palabras desde la segunda hasta la última. Esto se lleva a cabo mediante un objeto de la clase `LMDataLoader` que recibe como parámetro los tokens de nuestro dataset convertidos a números.

In [ ]:
nums200 = toks200.map(num)
dl = LMDataLoader(nums200)

Además el objeto que acabamos de construir parte el texto en bloques de tamaño 64 que serán usados para entrenar el modelo de lenguaje. 

In [ ]:
x,y = first(dl)
x.shape,y.shape

Podemos ver que el bloque "X" contiene una secuencia de tamaño 64, y el bloque "Y" la secuencia desplazada un token. 

In [ ]:
print(' '.join(num.vocab[o] for o in x[0][:20]))
print(' '.join(num.vocab[o] for o in y[0][:20]))

Todo este proceso que hemos explicado hasta ahora se realiza de manera automática por FastAI cuando construimos un modelo de lenguaje. 

### Modelo de lenguaje

Para crear un modelo de lenguaje en FastAI tenemos que:
1. Cargar los datos, y 
2. Entrenar el modelo

Para cargar los datos usamos la clase `DataBlock` del siguiente modo. 

In [42]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

Vamos a ver en detalle lo que hace la celda anterior. En primer lugar estamos definiendo una función lambda con el código:

``partial(get_text_files, folders=['train', 'test', 'unsup'])``

Es decir, el código anterior es equivalente a:

``lambda x: get_text_files(x, folders = ['train', 'test', 'unsup'])``

A continuación tenemos el ``DataBlock``

``DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
)``

al que le estamos diciendo que vamos a trabajar con datos de tipo textual (``TextBlock.from_folder(path, is_lm=True)``), que la manera de acceder a los ficheros de texto es a través de la función ``get_imdb`` que hemos definido anteriormente (``get_items=get_imdb``), y que el 10% del dataset se va a usar para evaluar el rendimiento del modelo (`splitter=RandomSplitter(0.1)`). El objeto `DataBlock` tiene además una serie de atributos que es necesario fijar, en concreto le tenemos que indicar el `path` donde están los ficheros, el tamaño del batch (en este caso 128), y la longitud de las secuencias (en este caso 80). 

Una vez que se han cargado los datos, podemos ver un batch. 

In [43]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj the script is idiotic , and the actors look and sound like porn movie rejects … \n\n xxmaj the horror is so boring with dumb effects and complete lack of suspense . xxmaj it 's amusing to see the main male character trying hard to be funny all through the movie with very sad rude one - liners . xxmaj you do n't really care for the characters as they are just a bunch of boring cannon fodder","xxmaj the script is idiotic , and the actors look and sound like porn movie rejects … \n\n xxmaj the horror is so boring with dumb effects and complete lack of suspense . xxmaj it 's amusing to see the main male character trying hard to be funny all through the movie with very sad rude one - liners . xxmaj you do n't really care for the characters as they are just a bunch of boring cannon fodder ."
1,"and his wicked sense of humour . day - lewis 's acting is not just physically convincing , in that it persuades us to believe in his character 's disability , but also emotionally and intellectually convincing , in that it brings out all these different facets of xxmaj christy 's character . xxmaj his xxmaj oscar was won in the teeth of some very strong opposition from the likes of xxmaj robin xxmaj williams and xxmaj kenneth xxmaj branagh","his wicked sense of humour . day - lewis 's acting is not just physically convincing , in that it persuades us to believe in his character 's disability , but also emotionally and intellectually convincing , in that it brings out all these different facets of xxmaj christy 's character . xxmaj his xxmaj oscar was won in the teeth of some very strong opposition from the likes of xxmaj robin xxmaj williams and xxmaj kenneth xxmaj branagh ,"


Ahora que los datos están listos podemos entrenar nuestro modelo de lenguaje.

### Fine tuning el modelo de lenguaje

El modelo de lenguaje es un modelo RNN que usa una arquitectura llamada *AWD_LSTM*. Para usar este modelo debemos construir un `Learner` del siguiente modo. 

In [44]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy,Perplexity()]).to_fp16()

En la celda anterior, estamos construyendo nuestro `Learner` al cual le indicamos de donde tiene que obtener los datos (`dls_lm`) y la arquitectura a usar (`AWD_LSTM`). Además le estamos indicando que aplique *Dropout* (`drop_mult=0.3`) y que mida el resultado del modelo usando la *accuracy* y una métrica conocida como *perplexity*. Por último, indicamos que el modelo se tiene que entrenar usando [precisión mixta](https://en.wikipedia.org/wiki/Half-precision_floating-point_format) (a la hora de representar reales) lo que acelera el proceso de entrenamiento. 

El proceso que vamos a seguir para entrenar nuestra red, es el mismo que el explicado en el entregable 2. 

Primero entrenamos solo las últimas capas del modelo (este paso puede llevar cierto tiempo).

In [46]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


RuntimeError: ignored

Y seguidamente descongelamos la red y la entrenamos entera.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

Como podéis ver estamos obteniendo una accuracy del 35% esto nos puede parecer bajo, pero daros cuenta que la tarea no es sencilla, estamos intentando adivinar la siguiente palabra de un texto, y hay múltiples posibilades, por lo que un modelo con una accuracy del 35% se puede considerar buena. 

Una vez que hemos entrenado el modelo, vamos a guardarlo para crear nuestro modelo de clasificación. Para el modelo de clasificación no va a ser necesaria la última capa del modelo de lenguaje (que es la que se encarga de predecir la siguiente palabra) por lo que vamos a guardar todo el modelo salvo esa última capa, a esto se le conoce como *encoder*. 

In [ ]:
learn.save_encoder('finetuned')

Con esto ya tenemos nuestro modelo de lenguaje creado, ahora podemos entrenar nuestro modelo de clasificación a partir de el. 

## Creando un clasificador

El proceso para crear un clasificador de texto es muy similar al que hemos seguido para crear un modelo de lenguaje.

En primer lugar tenemos que cargar los datos. Esto lo realizamos el siguiente modo.

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

Vamos a ver qué significa el código anterior:
- `blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock)` sirve para indicar que la entrada de nuesto modelo va a ser texto cargado a partir de los ficheros que se encuentran en path, y que la salida va a ser una clase (es decir que estamos en un problema de clasificación de texto). Notar que al contrario de lo que hacíamos antes, no pasamos el atributo ``is_lm=True`` esto se debe a que ya no estamos construyendo un modelo de lenguaje. Además, proporcionamos el vocabulario que queremos utilizar en lugar de pedir que lo construya. 
- `get_y = parent_label` sirve para indicar cómo vamos a indicar la etiqueta asociada con cada fichero. Recordar que las carpetas train y test tenían subcarpetas llamadas *pos* y *neg*. Con este código le indicamos que la etiqueta de cada fichero de texto va a ser el nombre de la carpeta que lo contiene. 
- `get_items=partial(get_text_files, folders=['train', 'test'])` indica de que subcarpetas se van a extraer los ficheros (recordar que nuestro path contiene otras carpetas adicionales).
- `splitter=GrandparentSplitter(valid_name='test')` indica que usaremos como conjunto de evaluación los ficheros de la carpeta *test*.
- `dataloaders(path, path=path, bs=128, seq_len=72)` configura los distintos parámetros del `DataBlock` de manera análoga a como lo hicimos en el caso del modelo del lenguaje. 

Podemos ver uno de los batches que se genera con el anterior `DataBlock` del siguiente modo.

In [ ]:
dls_clas.show_batch(max_n=3)

Ahora pasamos a construir nuestro `Learner` de manera similar a como lo hicimos anteriormente. 

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

Antes de empezar a entrenarlo, debemos cargar el *encoder* que guardamos anteriormente. 

In [ ]:
learn = learn.load_encoder('finetuned')

Ahora ya podemos entrenar nuestro clasificador. Comenzamos entrenando solo la última capa.

In [ ]:
learn.fit_one_cycle(1, 2e-2)

Ahora descongelamos dos capas y volvemos a entrenar. 

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

Podemos descongelar un poco más y seguir entrenando.

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

Y finalmente podemos entrenar el modelo completo.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

De este modo podemos lograr una accuracy del 94.3% que era el estado del arte hasta hace tres años. Es posible conseguir modelos mejores, pero eso se plantea como posible ampliación. 